#  The objective of this data analysis is to determine the number of patients who came to the clinic; determine various trends; and provide recommendations.

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime, timedelta
from scipy.stats import chi2_contingency
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
from urllib.request import urlretrieve

### Download the content from github and see the first 5 rows to get an idea what were working with. print the shape too to see the size of the dataframe

In [126]:
url = 'https://raw.githubusercontent.com/boltboihatesbananas/STI_Clinic/main/Data/STITestingDataset.csv'


df = pd.read_csv(url)
print(df.head(5), "\n\n")
print(df.shape)


  PatientNumber Sex  International Student (reg file)         Ethnicity  \
0      Patient1   M                                 0             Asian   
1      Patient1   M                                 0             Asian   
2      Patient1   M                                 0             Asian   
3      Patient1   M                                 0             Asian   
4     Patient10   F                                 0  Caucasian, White   

                           School Academic Class NCAA   CollectionDate  \
0                 Wharton Masters              G  NaN   1/10/2023 0:00   
1                 Wharton Masters              G  NaN   1/10/2023 0:00   
2                 Wharton Masters              G  NaN  2/23/2023 15:20   
3                 Wharton Masters              G  NaN  2/23/2023 15:20   
4  School Policy Practice Masters              G  NaN  1/19/2023 10:04   

                                     TestName        Result  
0  CHLAMYDIA TRACHOMATIS RNA, TMA, UROGENI

##### We have 5499 rows of 9 columns

In [3]:
df.dtypes

PatientNumber                               object
Sex                                         object
International Student (reg file)             int64
Ethnicity                                   object
School                                      object
Academic Class                              object
NCAA                                        object
CollectionDate                      datetime64[ns]
TestName                                    object
Result                                      object
dtype: object

##### Data types look ok. May want to change "International Student (reg file)" to an object.

### Examine data types, distribution, shape, duplicates, and nulls

In [4]:
df.describe()

,International Student (reg file),CollectionDate
count,5499.000000,5499
mean,0.250409,2023-03-15 08:08:27.659574528
min,0.000000,2023-01-03 09:23:00
25%,0.000000,2023-02-03 12:50:00
50%,0.000000,2023-03-14 15:30:00
75%,1.000000,2023-04-21 17:00:00
max,1.000000,2023-05-31 18:30:00
std,0.433288,NaN


#### Remove duplicates from the main dataframe but keep it for later analysis

#### Look for null values

In [5]:
df.isna().sum()

PatientNumber                          0
Sex                                    0
International Student (reg file)       0
Ethnicity                              0
School                                 0
Academic Class                         0
NCAA                                5386
CollectionDate                         0
TestName                               0
Result                                 0
dtype: int64

##### NCAA is only populated if the patient participates. We can fill the nulls with N/A for data completeness.

In [6]:
df['NCAA'].replace(np.nan, 'N/A', inplace=True)

df.isna().sum()

PatientNumber                       0
Sex                                 0
International Student (reg file)    0
Ethnicity                           0
School                              0
Academic Class                      0
NCAA                                0
CollectionDate                      0
TestName                            0
Result                              0
dtype: int64

##### All columns have no nulls now

#### Find duplicate values, store it in its own dataframe. Then remove them from the main dataframe.

In [7]:
duplicates = df[df.duplicated()]
df = df.drop_duplicates()

print("There are", duplicates.shape[0], "duplicates")
print("There are", df.shape[0], "non-duplicates")

There are 3 duplicates
There are 5496 non-duplicates


#### Obtain all the headers and turn it into a list. We can use this list to see what the distribution in the columns are like. However, for variables you expect to have a lot of variation like 'CollectionDate' you may want to drop it.

In [8]:
headers = list(df.columns)
headers.remove("CollectionDate")
print(headers)

['PatientNumber', 'Sex', 'International Student (reg file)', 'Ethnicity', 'School', 'Academic Class', 'NCAA', 'TestName', 'Result']


#### Now you can see the distribution

In [9]:
for header in headers:
    print(df[header].value_counts())
    print(df[header].value_counts(normalize=True).mul(100).round(1).astype(str) + '%')
    

PatientNumber
Patient1153    24
Patient88      20
Patient273     20
Patient439     20
Patient956     18
               ..
Patient237      2
Patient999      2
Patient625      1
Patient976      1
Patient1138     1
Name: count, Length: 1500, dtype: int64
PatientNumber
Patient1153    0.4%
Patient88      0.4%
Patient273     0.4%
Patient439     0.4%
Patient956     0.3%
               ... 
Patient237     0.0%
Patient999     0.0%
Patient625     0.0%
Patient976     0.0%
Patient1138    0.0%
Name: proportion, Length: 1500, dtype: object
Sex
F    2912
M    2538
U      34
X       6
N       6
Name: count, dtype: int64
Sex
F    53.0%
M    46.2%
U     0.6%
X     0.1%
N     0.1%
Name: proportion, dtype: object
International Student (reg file)
0    4120
1    1376
Name: count, dtype: int64
International Student (reg file)
0    75.0%
1    25.0%
Name: proportion, dtype: object
Ethnicity
Caucasian, White           1961
Asian                      1438
Multi ethnic/racial         872
Black, African-American  

##### Now we know what the distribution of the columns are we can start to look at what to focus on. We can also see areas to clean a little more. Within the columns we can consider whether the sum of each instance is significant enough for data analysis.

##### - Sex 
##### - International Students
##### - Ethnicity
##### - School but reclassify. All Wharton students to be 'Wharton' for example. Can analyze further with 'Academic Class'
##### - Academic class
##### - Test Name
##### - Result
##### - DateCollected could be parsed to date and time.

##### Now to reclassify School and DateCollected before Analysis

In [10]:
def map_school_category(school):
    if 'Wharton' in school:
        return 'Wharton'
    elif 'Arts' in school:
        return 'Arts and Sciences'
    elif 'LPS' in school:
        return 'LPS'
    elif 'Education' in school:
        return 'Education'
    elif 'Veterinary' in school:
        return 'Vet'
    elif 'Engineering' in school:
        return 'Engineering'
    elif 'BioMedical' in school:
        return 'Engineering'
    elif 'Nursing' in school:
        return 'Nursing'
    elif 'Law' in school:
        return 'Law'
    elif 'Dental' in school:
        return 'Dental'
    elif 'Design' in school:
        return 'Design'
    elif 'Communication' in school:
        return 'Communications'
    elif 'Medical' in school:
        return 'Medical'
    elif 'Policy Practice' in school:
        return 'Policy Practice'
    else:
        return school


# Create the new 'School_cat' column using the custom function
df['School_v2'] = df['School'].apply(map_school_category)

print(df['School_v2'].value_counts())
print(df['School'].value_counts())

School_v2
Arts and Sciences    1782
Wharton               934
Engineering           801
Education             355
Law                   314
Nursing               292
Medical               231
Dental                216
Design                214
Vet                   114
LPS                   105
Policy Practice        98
Communications         40
Name: count, dtype: int64
School
College Arts & Sciences               1339
Wharton Masters                        637
Graduate Arts Sciences                 443
BioMedical PhD                         411
Wharton Undergrad                      277
Law Schl                               264
Medical Schl                           231
Engineering Undergrad                  192
Design Professional                    190
Dental Medicine                        174
School of Education_PhD                168
Nursing Undergraduate                  165
Engineering Masters                    152
Education Masters/Doctorate            138
Veterinary Schl  

In [11]:
df.loc[:, 'Date'] = df['CollectionDate'].dt.date
df['Date'] = pd.to_datetime(df["Date"])
df['CollectionDate'] = pd.to_datetime(df['CollectionDate'])
df['Time'] = df['CollectionDate'].dt.time
df['Hour'] = df['Time'].apply(lambda x: x.hour)
df['Day'] = df['Date'].dt.day_name()
df['Month'] = df['Date'].dt.month


In [12]:
df[['Date', 'Time', 'Hour', 'Day', 'Month']].head()

,Date,Time,Hour,Day,Month
0,2023-01-10,00:00:00,0,Tuesday,1
1,2023-01-10,00:00:00,0,Tuesday,1
2,2023-02-23,15:20:00,15,Thursday,2
3,2023-02-23,15:20:00,15,Thursday,2
4,2023-01-19,10:04:00,10,Thursday,1


##### Now we need to make sure that what we made is valid with another round of checks. Start with null values. 

##### Then we can check the distribution.

In [13]:
df[['Date', 'Time', 'Hour', 'Day', 'Month']].isnull().sum()

Date     0
Time     0
Hour     0
Day      0
Month    0
dtype: int64

In [14]:
print("The distribution for Hours are", df['Hour'].value_counts().sort_index(ascending=True))
print("The distribution for Days are", df['Day'].value_counts().sort_index(ascending=True))
print("The distribution for Months are", df['Month'].value_counts().sort_index(ascending=True))

The distribution for Hours are Hour
0     374
1      10
2       2
3       6
4      12
5       4
7       2
8      14
9     553
10    764
11    832
12    293
13    395
14    644
15    643
16    622
17    158
18    103
19     59
20      6
Name: count, dtype: int64
The distribution for Days are Day
Friday        815
Monday       1083
Saturday       87
Sunday          4
Thursday     1269
Tuesday      1141
Wednesday    1097
Name: count, dtype: int64
The distribution for Months are Month
1    1227
2    1054
3    1092
4    1029
5    1094
Name: count, dtype: int64
